# Core Concepts

- Bayes Theorem: $$P(A|B) = \frac{P(B|A)P(A)}{P(B)}$$ $$P(y|X) = \frac{P(X|y)P(y)}{P(X)}$$
- Assumption: all features are mutually independent: $$P(y|X) = \frac{P(x_1|y)P(x_2|y)...P(x_n|y)P(y)}{P(X)}$$
    - $P(x_n|y)$: conditional probabillity -> model with Gaussian
    - $P(y)$: prior of y -> frequency
    - $P(y|X)$: posterior
    - NaiveBayes doesn't come with weights & biases for optimization, instead the log-likelihoods and the conditional probability Gaussian model behind can be modeled parametrically
- classification - making prediction: $$y = argmax_y P(y|X) = argmax_y  \frac{P(x_1|y)P(x_2|y)...P(x_n|y)P(y)}{P(X)}$$
    - all probabilities between 0 and 1, cumulative multiplication can ended up be really small numbers
    - take logprobs instead: $$y = argmax_y log(P(x_1|y)) ... + log(P(x_n|y)) + log(P(y))$$ 

# Code

In [8]:
import numpy as np

class NaiveBayes:

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self._classes = np.unique(y)
        n_classes = len(self._classes)

        # init mean, var, priors
        self._mean = np.zeros((n_classes, n_features), dtype = np.float64)
        self._var = np.zeros((n_classes, n_features), dtype = np.float64)
        self._priors = np.zeros(n_classes, dtype = np.float64)

        for c in self._classes:
            X_c = X[y==c]
            self._mean[c, :] = X_c.mean(axis=0)
            self._var[c, :] = X_c.var(axis=0)
            self._priors[c] = X_c.shape[0] / float(n_samples)

    
    def predict(self, X):
        y_pred = [self._predict(x) for x in X]
        return y_pred

    def _predict(self, x):
        posteriors = []

        for idx, c in enumerate(self._classes):
            prior = np.log(self._priors[idx])
            class_conditional = np.log(self._pdf(idx, x))
            posterior = np.sum(class_conditional) + prior
            posteriors.append(posterior)
        
        return self._classes[np.argmax(posteriors)]
    
    def _pdf(self, class_idx, x):
        mean = self._mean[class_idx]
        var = self._var[class_idx]
        numerator = np.exp( -(x-mean)**2/(2*var))
        denominator = np.sqrt(2 * np.pi * var)
        return numerator / denominator

# Test

In [9]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

def accuracy(y_true, y_pred):
    return np.sum(y_true == y_pred) / len(y_true)

X, y = datasets.make_classification(n_samples=1000, n_features=10, n_informative=2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

nb = NaiveBayes()
nb.fit(X_train, y_train)
y_pred = nb.predict(X_test)

print("Accuracy:", accuracy(y_test, y_pred))

Accuracy: 0.81
